## Part 2: FourSquare and Yelp APIs
This notebook makes use of the City Bikes data collected for Part 1. It is loaded from a CSV file before accesing the APIs.

In [ ]:
# import required libraries
import pandas as pd
import os # use this to access environment variables
import requests # this will be used to call the APIs
import json

# get the Foursquare environment variable from the OS file
FOURSQUARE_KEY = os.getenv('FOURSQUARE_API_KEY')

# the city_bikes.ipnb notebook created a CSV for it's output, and this is read into a dataframe here
city_bikes_df = pd.read_csv('../data/city_bikes.csv')
city_bikes_df.head()

,id,name,latitude,longitude,timestamp,free_bikes,empty_slots,extra.uid,extra.number,extra.slots,extra.bike_uids,extra.virtual
0,066c99293af108ece27d9b0436c30cc4,Riverside Museum,55.865683,-4.305367,2025-07-01T19:18:19.376003+00:00Z,11,0,19738176,8416,6,"['47041', '823506', '46976', '46961', '46910',...",False
1,0a106cbc148d5a0c2535b51c1dbe3b4d,Parkhead,55.846524,-4.197475,2025-07-01T19:18:19.377604+00:00Z,6,0,150125231,8470,8,"['82316', '840047', '46722', '46457', '47968',...",False
2,0cc8c09950e1435ee7782478ed292fef,Tollcross International Swimming Centre,55.844250,-4.176167,2025-07-01T19:18:19.376951+00:00Z,6,0,55599921,8200,6,"['828253', '46967', '823632', '828964', '82360...",False
3,0e94d2ad012bff0cf23497963ff5fd77,Shields Road,55.845807,-4.275232,2025-07-01T19:18:19.375677+00:00Z,1,7,7145152,8403,8,['46992'],False
4,101c6cd7749f373507e9444f5222b7f2,Shawlands Shopping Centre (Kilmarnock Road),55.829057,-4.282675,2025-07-01T19:18:19.375152+00:00Z,1,6,3812605,8453,8,['46830'],False


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [3]:
def get_venues_fs(latitude, longitude, radius, api_key, categories=None):
    """
    Get venues from foursquare with a specified place type and coordinates.
    Args:
        latitude (float): latitude for query (must be combined with longitude)
        longitude (float): longitude for query (must be combined with latitude)
        api_key (str): foursquare API to use for query
        categories (str) : Foursquare-recognized place type. If not passed no place_type will be specified. Separate ids with commas
    
    Returns:
        number of venues (POI) of the specified category
    """
    
    url = "https://places-api.foursquare.com/places/search"
    
    params = {
        "ll":f"{latitude},{longitude}",
        "radius": radius,
        "limit": 50
    }

    if categories:
        params["categories"] = categories

    headers = {
        "accept": "application/json",
        "X-Places-Api-Version": "2025-06-17",
        "authorization": "Bearer "+api_key
    }

    response = requests.get(url, params=params, headers=headers)

    # print(response.text)
    # print(json.loads(response.text))
    # print(response.json().get("results", []))
    
    # The function returns the venues for the specified area
    return response.json().get("results", [])



Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame


In [16]:

# Fixed parameters used in function call
API_KEY = os.getenv('FOURSQUARE_API_KEY')
RADIUS = 1000  # meters
CATEGORIES = "4bf58dd8d48988d116941735" # The category code for bars and similar businesses. 
# I'm not sure that this is working correctly, as there appear to be a lot of entries that are likely not bars.
# It could just be that I've chosen a FourSquare category that's very broad.

# Call the function for each row. 
# This adds the information retrieved from the Foursquare API to a new dataframe, with bike station data.

fs_dfs = []  # List to collect individual DataFrames

# iterate through all the bike station locations in the city_bikes_df dataframe.
for i in range(len(city_bikes_df)):
    row = city_bikes_df.iloc[i]
    lat = row['latitude']
    lon = row['longitude']
    bike_id = row['id']
    free_bikes = row['free_bikes']

# Call the function to retrieve FourSquare POI data from around bike stations
    fs_json = get_venues_fs(
        latitude= lat,
        longitude= lon,
        radius=RADIUS,
        api_key=API_KEY,
        categories=CATEGORIES
    )

# Flatten the json POI data returned by FourSquare API
    fs_venues_df = pd.json_normalize(
        fs_json,
        record_path=['categories'],
        meta=[
            'fsq_place_id',
            'latitude',
            'longitude',
            'distance',
            'email',
            'link',
            ['location', 'address'],
            ['location', 'locality']
        ],
        errors = 'ignore'
    )
# Add bike station location and ID to THIS batch of POIs before appending
    if not fs_venues_df.empty:
        fs_venues_df['bike_lat'] = lat
        fs_venues_df['bike_lon'] = lon
        fs_venues_df['bike_id'] = bike_id
        fs_venues_df['free_bikes'] = free_bikes
        fs_dfs.append(fs_venues_df)

# Concatenate all DataFrames at once
if fs_dfs:
    fs_df = pd.concat(fs_dfs, ignore_index=True)
else:
    fs_df = pd.DataFrame()

fs_df


,fsq_category_id,name,short_name,plural_name,icon.prefix,icon.suffix,fsq_place_id,latitude,longitude,distance,email,link,location.address,location.locality,bike_lat,bike_lon,bike_id,free_bikes
0,4bf58dd8d48988d181941735,Museum,Museum,Museums,https://ss3.4sqi.net/img/categories_v2/arts_en...,.png,4c8a3ba4a92fa093be438fbf,55.865392,-4.306417,73,museums@glasgowlife.org.uk,/places/4c8a3ba4a92fa093be438fbf,Glasgow Harbour,Glasgow,55.865683,-4.305367,066c99293af108ece27d9b0436c30cc4,11
1,4bf58dd8d48988d190941735,History Museum,History Museum,History Museums,https://ss3.4sqi.net/img/categories_v2/arts_en...,.png,4c8a3ba4a92fa093be438fbf,55.865392,-4.306417,73,museums@glasgowlife.org.uk,/places/4c8a3ba4a92fa093be438fbf,Glasgow Harbour,Glasgow,55.865683,-4.305367,066c99293af108ece27d9b0436c30cc4,11
2,4bf58dd8d48988d16d941735,Café,Café,Cafés,https://ss3.4sqi.net/img/categories_v2/food/cafe_,.png,4eaaa5f477c850207ed9fee8,55.86561,-4.305612,17,NaN,/places/4eaaa5f477c850207ed9fee8,Riverside Museum,Glasgow,55.865683,-4.305367,066c99293af108ece27d9b0436c30cc4,11
3,4bf58dd8d48988d1e2931735,Art Gallery,Art Gallery,Art Galleries,https://ss3.4sqi.net/img/categories_v2/arts_en...,.png,4b86d7fcf964a52018a131e3,55.864562,-4.299637,378,work@swg3.tv,/places/4b86d7fcf964a52018a131e3,100 Eastvale Pl.,Glasgow,55.865683,-4.305367,066c99293af108ece27d9b0436c30cc4,11
4,4bf58dd8d48988d1e5931735,Music Venue,Music Venue,Music Venues,https://ss3.4sqi.net/img/categories_v2/arts_en...,.png,4b86d7fcf964a52018a131e3,55.864562,-4.299637,378,work@swg3.tv,/places/4b86d7fcf964a52018a131e3,100 Eastvale Pl.,Glasgow,55.865683,-4.305367,066c99293af108ece27d9b0436c30cc4,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6163,4bf58dd8d48988d116941735,Bar,Bar,Bars,https://ss3.4sqi.net/img/categories_v2/nightli...,.png,4b597c69f964a520d18928e3,55.864598,-4.263195,188,info@thebunkerbar.com,/places/4b597c69f964a520d18928e3,193-199 Bath St,Glasgow,55.865063,-4.260291,faf9e8803c8d80741d74f98cb9ba8337,2
6164,4bf58dd8d48988d11f941735,Night Club,Night Club,Night Clubs,https://ss3.4sqi.net/img/categories_v2/nightli...,.png,4b597c69f964a520d18928e3,55.864598,-4.263195,188,info@thebunkerbar.com,/places/4b597c69f964a520d18928e3,193-199 Bath St,Glasgow,55.865063,-4.260291,faf9e8803c8d80741d74f98cb9ba8337,2
6165,4d4b7105d754a06374d81259,Restaurant,Restaurant,Restaurants,https://ss3.4sqi.net/img/categories_v2/food/de...,.png,4b597c69f964a520d18928e3,55.864598,-4.263195,188,info@thebunkerbar.com,/places/4b597c69f964a520d18928e3,193-199 Bath St,Glasgow,55.865063,-4.260291,faf9e8803c8d80741d74f98cb9ba8337,2
6166,4bf58dd8d48988d110941735,Italian Restaurant,Italian,Italian Restaurants,https://ss3.4sqi.net/img/categories_v2/food/it...,.png,4bfc03c91be376b09907f8b4,55.865457,-4.264588,271,NaN,/places/4bfc03c91be376b09907f8b4,305 Sauchiehall St,Glasgow,55.865063,-4.260291,faf9e8803c8d80741d74f98cb9ba8337,2


In [ ]:
# Save the parsed dataframe to a CSV file for further processing in the later parts of the exercise.
fs_df.to_csv('../data/foursquare_poi_data.csv', index=False)


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [8]:
import requests

def get_venues_yelp(latitude, longitude, radius, api_key):
    """
    Get venues from Yelp within a specified location.
    Args:
        latitude (float): latitude for query
        longitude (float): longitude for query
        radius (int): how far to search from central point
        api_key (str): Yelp API key to use for query
        
    Returns:
        venues (POI) information
    """
    
    url = "https://api.yelp.com/v3/businesses/search?"
    # Full query example "https://api.yelp.com/v3/businesses/search?latitude=55.865392&longitude=-4.306417&radius=1000&sort_by=best_match&limit=20"

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "radius": radius,
        "sort_by": 'best_match',
        "limit": 20  #A limit placed on how many results to return for each request.
    }

    headers = {
        "accept": "application/json",
        "authorization": f"Bearer {api_key}"
    }

    response = requests.get(url, params=params, headers=headers) # Make the appropriate request to Yelp using the above params/headers.
    response.raise_for_status()  # Check for HTTP errors

    # print(response.text)
    # print(json.loads(response.text))
    # print(response.json().get("businesses", []))
    
    # The function returns the venues for the specified area
    return response.json().get("businesses", [])


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

In [18]:
# Fixed parameters
YELP_KEY = os.getenv('YELP_KEY')  # get the YELP environment variable from the OS file
RADIUS = 1000  # meters

# Call the function for each row. 
# This adds the information retrieved from the Yelp API to a new dataframe, with bike station data.

dfs = []  # Initialize an empty list as temporary place for dataframes to be concatenated

# iterate through all the bike station locations in the city_bikes_df dataframe.
for i in range(len(city_bikes_df)): 
    lat = city_bikes_df['latitude'].iloc[i]
    lon = city_bikes_df['longitude'].iloc[i]
    bike_id = city_bikes_df['id'].iloc[i]
    free_bikes = city_bikes_df['free_bikes'].iloc[i]

# Call the function to retrieve Yelp POI data from around bike stations
    poi_json = get_venues_yelp(lat, lon, RADIUS, YELP_KEY)

# Flatten the json POI data returned by Yelp API
    yelp_venues_df = pd.json_normalize(
        poi_json,
        record_path="categories",       # Path to expand nested lists
        meta=[
            "id", 
            "name", 
            "rating",
            ["coordinates","latitude"],
            ["coordinates","longitude"],
            ["location", "address1"], 
            ["location", "city"], 
            "distance"],  # Top-level columns
        meta_prefix="yelp_",
        record_prefix="category_",
        errors = 'ignore'
    )
# Add bike station metadata to ALL rows in yelp_venues_df
    yelp_venues_df['bike_lat'] = lat
    yelp_venues_df['bike_lon'] = lon
    yelp_venues_df['bike_id'] = bike_id
    yelp_venues_df['free_bikes'] = free_bikes
    dfs.append(yelp_venues_df)

yelp_df = pd.concat(dfs, ignore_index=True)
yelp_df


,category_alias,category_title,yelp_id,yelp_name,yelp_rating,yelp_coordinates.latitude,yelp_coordinates.longitude,yelp_location.address1,yelp_location.city,yelp_distance,bike_lat,bike_lon,bike_id,free_bikes
0,tapasmallplates,Tapas/Small Plates,_GHNOOJhhjLegiCvzf5V0Q,Ox And Finch,4.7,55.865654,-4.284692,920 Sauchiehall Street,Glasgow,1290.029875,55.865683,-4.305367,066c99293af108ece27d9b0436c30cc4,11
1,scottish,Scottish,_GHNOOJhhjLegiCvzf5V0Q,Ox And Finch,4.7,55.865654,-4.284692,920 Sauchiehall Street,Glasgow,1290.029875,55.865683,-4.305367,066c99293af108ece27d9b0436c30cc4,11
2,british,British,jGLzc9eIUAnfto_5oZ8BMQ,Number 16 Restaurant,4.6,55.87081,-4.298839,16 Byres Road,Glasgow,700.694335,55.865683,-4.305367,066c99293af108ece27d9b0436c30cc4,11
3,bars,Bars,rmaH1My396rCYEnL8XQtPg,Ubiquitous Chip,4.3,55.874915,-4.293247,8-12 Ashton Lane,Glasgow,1274.920333,55.865683,-4.305367,066c99293af108ece27d9b0436c30cc4,11
4,scottish,Scottish,rmaH1My396rCYEnL8XQtPg,Ubiquitous Chip,4.3,55.874915,-4.293247,8-12 Ashton Lane,Glasgow,1274.920333,55.865683,-4.305367,066c99293af108ece27d9b0436c30cc4,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4159,steak,Steakhouses,4B7sGFSBSDjiZMBrYG6AYg,Alston Bar & Beef,4.5,55.860346,-4.257653,Glasgow Central Station,Glasgow,549.718337,55.865063,-4.260291,faf9e8803c8d80741d74f98cb9ba8337,2
4160,cocktailbars,Cocktail Bars,4B7sGFSBSDjiZMBrYG6AYg,Alston Bar & Beef,4.5,55.860346,-4.257653,Glasgow Central Station,Glasgow,549.718337,55.865063,-4.260291,faf9e8803c8d80741d74f98cb9ba8337,2
4161,pubs,Pubs,Oal3XEeDwWeu21lJb-lVgg,Mini Grill,4.7,55.865029,-4.265696,244a Bath Street,Glasgow,337.273209,55.865063,-4.260291,faf9e8803c8d80741d74f98cb9ba8337,2
4162,scottish,Scottish,Oal3XEeDwWeu21lJb-lVgg,Mini Grill,4.7,55.865029,-4.265696,244a Bath Street,Glasgow,337.273209,55.865063,-4.260291,faf9e8803c8d80741d74f98cb9ba8337,2


In [ ]:
# Save the parsed dataframe to a CSV file for further processing in the later parts of the exercise.
yelp_df.to_csv('../data/yelp_poi_data.csv', index=False)

# Comparing Results

#### Which API provided you with more complete data? Provide an explanation. 

*Answer*: The Yelp API appears to give more complete venue information. Ratings were easy to find and process. Yelp also gives opening hours, links to things like menus, etc. By contrast, the FourSquare API seems to be about its internal classifications. The classification information was prominent. FourSquare produced more points (around 6000) of interest than Yelp (around 4000).

#### Get the top 10 restaurants according to their rating

In [11]:
# The top-rated venues on Yelp, returned by the city bikes locations
# (I couldn't find similar ratings in the FourSquare response)
yelp_df.sort_values(by='yelp_rating', ascending=False).head(10)

,category_alias,category_title,yelp_id,yelp_name,yelp_rating,yelp_coordinates.latitude,yelp_coordinates.longitude,yelp_location.address1,yelp_location.city,yelp_distance,bike_lat,bike_lon,bike_id
687,hotdogs,Fast Food,Il_x6PMmzJUnNHWikPvu0w,M&S Simply Food,5.0,55.864662,-4.334974,South Glasgow University Hospital,Glasgow,365.745487,55.862838,-4.341885,3860eb7809fcced150d10d78ca2a165c
1837,pubs,Pubs,ZB3mVUeMrJ4JaSdUHHT4dA,Gartocher Bar,5.0,55.850502,-4.154971,1618 Shettleston Road,Glasgow,1671.340265,55.840336,-4.135251,7fd473d65bb4ead7217d778a38d56dc2
644,bars,Bars,bp_fzGEY7NQnH0gOq6Z27g,Abarcrombys Cafe Bar,5.0,55.876879,-4.344608,180 Dumbarton Road,Glasgow,214.953151,55.875431,-4.342325,2ee7f8a8347b8ee388d6beeef94c837d
645,cafes,Cafes,bp_fzGEY7NQnH0gOq6Z27g,Abarcrombys Cafe Bar,5.0,55.876879,-4.344608,180 Dumbarton Road,Glasgow,214.953151,55.875431,-4.342325,2ee7f8a8347b8ee388d6beeef94c837d
654,cupcakes,Cupcakes,Yl1Quh6mG2nvWhaCAMMj1A,Cupcakes For You & West End Vintage,5.0,55.874676,-4.339886,1251 Dumbarton Road,Glasgow,214.745327,55.875431,-4.342325,2ee7f8a8347b8ee388d6beeef94c837d
667,cafes,Cafes,CoJYmpB6BV0OmLnREot3pA,Harbour Cafe,5.0,55.872826,-4.34021,719 South Street,Glasgow,314.985284,55.875431,-4.342325,2ee7f8a8347b8ee388d6beeef94c837d
1872,lakes,Lakes,Il96dNIpp49is-WIRG-cvw,The River Cart,5.0,55.818231,-4.26445,Cathcart,Cathcart,143.084625,55.817338,-4.262688,7fde42277f0af769fa2844ac44fb18bb
1871,localflavor,Local Flavor,Il96dNIpp49is-WIRG-cvw,The River Cart,5.0,55.818231,-4.26445,Cathcart,Cathcart,143.084625,55.817338,-4.262688,7fde42277f0af769fa2844ac44fb18bb
1187,pakistani,Pakistani,FpXxXLgw5-om9lNd_6sI2w,Ambala Restaurant,5.0,55.843264,-4.269499,11 Forth Street,Glasgow,198.300432,55.841494,-4.269701,5c9021a0e68ddd0d3e47cc90e59c77c8
688,sandwiches,Sandwiches,Il_x6PMmzJUnNHWikPvu0w,M&S Simply Food,5.0,55.864662,-4.334974,South Glasgow University Hospital,Glasgow,365.745487,55.862838,-4.341885,3860eb7809fcced150d10d78ca2a165c


The list of ratings isn't great because there are numerous duplicated places in the list. Duplicates appear to be the result of places appearing in multiple categories.